# Discovery File Validation Notebook
### Description: 
This notebook is intended to automate validation for discovery files
and provide questions/examples for the client about the provided data files

In [32]:
import os
import numpy as np
import pandas as pd


## Read in files from data directory and create Dataframes for each file

In [36]:
file_types = [
    'companies',
    'customers', 
    'drivers', 
    'moves', 
    'orders', 
    'stops', 
    'tractors',
    'trailers'
    ]

data_path = '../data/amx/'

files = []
data = {}
files_received = []
for i in os.listdir(data_path):
    if os.path.isfile(os.path.join(data_path,i)):
        files.append(i)

for file in files:
    file_type = file.split('-')[0]
    data[file_type] = pd.read_csv(data_path + file, sep='\t')

for file in file_types:
    try:
        data[file]
        files_received.append(file)
    except KeyError:
        print(f'The {file} file is missing from the data folder')

for file in files_received:
    print(f"{file} - {type(data[file])}")

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


companies - <class 'pandas.core.frame.DataFrame'>
customers - <class 'pandas.core.frame.DataFrame'>
drivers - <class 'pandas.core.frame.DataFrame'>
moves - <class 'pandas.core.frame.DataFrame'>
orders - <class 'pandas.core.frame.DataFrame'>
stops - <class 'pandas.core.frame.DataFrame'>
tractors - <class 'pandas.core.frame.DataFrame'>
trailers - <class 'pandas.core.frame.DataFrame'>


/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Validation for each file

#### Drivers File

In [37]:
driver_df = data['drivers']

print(driver_df.info(verbose = True))
print("\n")
print("TYPE_OF - COUNTS")
print(driver_df['type_of'].value_counts())
print("\n")
print('COUNTRY_HOS_RULES - COUNTS')
print(driver_df['country_hos_rules'].value_counts())
print("\n")
print('GROUP_ID - COUNTS')
print(driver_df['group_id'].value_counts())
print("\n")
print('DRIVERS WITH ONDUTY HOURS OUTSIDE NORM')
avl_onduty_hours_df = driver_df[(driver_df['avl_onduty_hours'] < 0) | (driver_df['avl_onduty_hours'] > 14)]
print(avl_onduty_hours_df[['avl_onduty_hours']])
print('\n')
print('DRIVERS WITH AVALABLE DRIVER HOURS OUTSIDE NORM')
avl_drive_hours_df = driver_df[(driver_df['avl_drive_hours'] < 0) | (driver_df['avl_drive_hours'] > 11)]
print(avl_drive_hours_df[['avl_drive_hours']])
print('\n')
print("TERMINATED DRIVERS WITH TRUE ACTIVE STATUS")
terminated_drivers_df = driver_df[(driver_df['termination_date'] > driver_df['hire_date'])]
print(terminated_drivers_df[['is_active', 'termination_date', 'hire_date']].loc[terminated_drivers_df['is_active'] == True])
print('\n')
print("REHIRED DRIVERS WITH FALSE ACTIVE STATUS")
rehired_drivers_df = driver_df[(driver_df['termination_date'] < driver_df['hire_date'])]
print(rehired_drivers_df[['is_active', 'termination_date', 'hire_date']].loc[rehired_drivers_df['is_active'] == False])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   is_active            503 non-null    bool   
 1   latitude             503 non-null    float64
 2   type_of              503 non-null    object 
 3   last_home_date       24 non-null     object 
 4   __currentMovementId  503 non-null    int64  
 5   state                503 non-null    object 
 6   termination_date     287 non-null    object 
 7   hazmat_certified     503 non-null    bool   
 8   event_date           492 non-null    object 
 9   country_hos_rules    503 non-null    object 
 10  first_name           502 non-null    object 
 11  group_id             461 non-null    object 
 12  fleet_manager        441 non-null    object 
 13  company_id           503 non-null    object 
 14  license_date         503 non-null    object 
 15  hire_date            503 non-null    obj

#### Moves File

In [4]:
moves_df = data['moves']

print(moves_df.info(verbose = True))
print("\n")
print('MOVE_STATUS - COUNTS')
print(moves_df['move_status'].value_counts())
print("\n")
print('LOADED - COUNTS')
print(moves_df['loaded'].value_counts())
print("\n")
print('ORDER_ID - BLANKS')
order_id_df = moves_df['order_id'].isnull()
print(order_id_df)
print("\n")
print('PRORATED_REVENUE - NEGATIVE')
prorated_revenue_df = moves_df[(moves_df['prorated_revenue'] < 0)]
print(prorated_revenue_df[['prorated_revenue']])
print("\n")
print('BROKERAGE - COUNTS')
print(moves_df['brokerage'].value_counts())
print("\n")
print('MOVE_DISTANCE - OUT OF BOUNDS (<0 OR >4000)')
move_distance_df = moves_df[(moves_df['move_distance'] < 0)|(moves_df['move_distance'] > 4000)]
print(move_distance_df[['move_distance']])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209261 entries, 0 to 209260
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   move_status       209261 non-null  object 
 1   is_preassignment  133450 non-null  object 
 2   loaded            209261 non-null  object 
 3   order_id          197736 non-null  float64
 4   prorated_revenue  129573 non-null  float64
 5   trailer_id        137152 non-null  object 
 6   id                209261 non-null  int64  
 7   driver_id         133450 non-null  object 
 8   brokerage         209261 non-null  bool   
 9   move_distance     209237 non-null  float64
 10  tractor_id        134556 non-null  object 
dtypes: bool(1), float64(3), int64(1), object(6)
memory usage: 16.2+ MB
None


MOVE_STATUS - COUNTS
D    208684
P       407
A       170
Name: move_status, dtype: int64


LOADED - COUNTS
L    129578
E     79683
Name: loaded, dtype: int64


ORDER_ID - BLANKS
0       

#### Orders File

In [5]:
orders_df = data['orders']

print(orders_df.info(verbose = True))
print("\n")
print('STATUS - COUNTS')
print(orders_df['status'].value_counts())
print("\n")
print('COMMODITY_ID - COUNTS')
print(orders_df['commodity_id'].value_counts())
print("\n")
print('REVENUE_CODE_ID - COUNTS')
print(orders_df['revenue_code_id'].value_counts())
print("\n")
print('CUSTOMER_ID - BLANKS')
print(orders_df[orders_df['customer_id'].isnull()])
print('\n')
print('CUSTOMER_ID - COUNTS TOP 10')
print(orders_df['customer_id'].value_counts().head(10))
print("\n")
print('CUSTOMER_ID - BL')
print(orders_df['customer_id'].value_counts())
print("\n")
print('ORDERED_DATE - BLANKS')
print(orders_df[orders_df['ordered_date'].isnull()])
print("\n")
print('BILL_DISTANCE - OUT OF BOUNDS (<0 OR >4000)')
print(orders_df[(orders_df['bill_distance'] < 0)|(orders_df['bill_distance'] > 4000)])
print("\n")
print('FREIGHT_CHARGE - NEGATIVE')
print(orders_df[orders_df['freight_charge'] < 0])
print("\n")
print('OTHERCHARGETOTAL - NEGATIVE')
print(orders_df[orders_df['otherchargetotal'] < 0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119634 entries, 0 to 119633
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   pallets_how_many  7897 non-null    float64
 1   status            119634 non-null  object 
 2   bill_distance     119633 non-null  float64
 3   freight_charge    119634 non-null  float64
 4   entered_user_id   119634 non-null  object 
 5   commodity_id      89077 non-null   object 
 6   weight            119494 non-null  float64
 7   id                119634 non-null  int64  
 8   revenue_code_id   119634 non-null  object 
 9   otherchargetotal  119634 non-null  float64
 10  customer_id       119634 non-null  object 
 11  ordered_date      119634 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 11.0+ MB
None


STATUS - COUNTS
D    119051
P       440
A       137
V         6
Name: status, dtype: int64


COMMODITY_ID - COUNTS
FOOD         17619
PA           

#### Stops File

In [29]:
stops_df = data['stops']

print(stops_df.info(verbose = True))
print("\n")
print('ACTUAL_ARRIVAL - BLANKS')
actual_arrival_df = stops_df[stops_df['actual_arrival'].isnull()]
print(actual_arrival_df[['actual_arrival']])
print("\n")
print('ACTUAL_DEPARTURE - BLANKS')
actual_departure_df = stops_df[stops_df['actual_departure'].isnull()]
print(actual_departure_df[['actual_departure']])
print("\n")
print('STOP_TYPE - COUNTS')
print(stops_df['stop_type'].value_counts())
print("\n")
print('SCHED_ARRIVE_EARLY - BLANKS')
sched_arrive_early_df = stops_df[stops_df['sched_arrive_early'].isnull()]
print(sched_arrive_early_df[['sched_arrive_early']])
print("\n")
print('SCHED_ARRIVE_LATE - BLANKS')
sched_arrive_late_df = stops_df[stops_df['sched_arrive_late'].isnull()]
print(sched_arrive_late_df[['sched_arrive_late']])
print("\n")
print('ZIP_CODE - BLANKS')
zip_code_blank_df = stops_df[stops_df['zip_code'].isnull()]
print(zip_code_blank_df[['zip_code']])
print("\n")
print('ZIP_CODE - INCORRECT FORMAT - Too Short')
zip_code_short_df = stops_df[(stops_df['zip_code'].str.len() < 5)]
print(zip_code_short_df[['zip_code']])
print("\n")
print('ZIP_CODE - INCORRECT FORMAT - Too Long')
zip_code_partition_hyphen_df = stops_df['zip_code'].str.partition('-')
zip_code_long_df = zip_code_partition_hyphen_df[(zip_code_partition_hyphen_df[0].str.len() > 5)]
zip_code_long_df = zip_code_long_df.rename(columns={0: 'zip_code'})
print(zip_code_long_df[['zip_code']])
print("\n")
print('ZIP_CODE - INCORRECT FORMAT - Non-numeric Characters')
zip_code_no_hyphen_df = stops_df['zip_code'].str.replace('-', '')
zip_code_alpha_df = zip_code_no_hyphen_df[zip_code_no_hyphen_df.str.isnumeric() == False]
print(zip_code_alpha_df)
print("\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427036 entries, 0 to 427035
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   zip_code            426388 non-null  object 
 1   address             415482 non-null  object 
 2   state               427036 non-null  object 
 3   order_id            267670 non-null  float64
 4   actual_arrival      426303 non-null  object 
 5   city_name           427036 non-null  object 
 6   driver_load_unload  427036 non-null  object 
 7   actual_departure    426225 non-null  object 
 8   sched_arrive_late   309235 non-null  object 
 9   address2            61599 non-null   object 
 10  id                  427036 non-null  object 
 11  sched_arrive_early  427036 non-null  object 
 12  move_id             427036 non-null  int64  
 13  stop_type           427036 non-null  object 
 14  movement_sequence   427036 non-null  int64  
 15  order_sequence      377007 non-nul

#### Tractors File

#### Trailers File